In [ ]:
from core import AIDeFiCore
from query import sqlQueries
import pandasql as ps
import statistics as stat

In [ ]:
from classes import ethData
ethData = ethData.ethData()
BSCSCAN_API_KEY = 'I6ZZ57GWAS39M85QC4AJFC43Y5DUA8IU8I'
#ethData.API_String('BSCSCAN', 'TXLIST', ['0xB0B924C4a31b7d4581a7F78F57ceE1E65736Be1D', 0, 9999999999999, BSCSCAN_API_KEY])
ethData.analyzeTokenCustomers(BSCSCAN_API_KEY)

In [ ]:
ethData.getMaps(0, 'TOKEN')

In [ ]:
from classes import ethData
ethData = ethData.ethData()
ETHERSCAN_API_KEY = 'EWYSGQ5PEK84PRCNSCKHHZWJIWWKYYABCE'
ethData.analyzeDexCustomers(ETHERSCAN_API_KEY)

In [ ]:
ethData.importAllDexTransactions(ETHERSCAN_API_KEY, 'ETHERSCAN', 'TXLIST', 'DEX', 'MAX', 0, 2) # 24 hours
ethData.importTransactionsFromDexCustomers(ETHERSCAN_API_KEY, 'ETHERSCAN', 'TXLIST', 'DEX')
ethData.mergeFiles('DEX')
ethData.reportDexCustomers('DEX')

In [ ]:
transactionsFromUsersDex = pd.read_csv('export/CUSTOMERS_TRANSACTIONS_FROM_DEX.csv')
transactionsFromUsersDex.set_index(transactionsFromUsersDex['from'])
customerList = pd.read_csv('export/CUSTOMERS_LIST_FROM_DEX.csv')
customerList.rename(columns={'from':'fromAddress'}, inplace=True)
transactionsFromUsersDex.rename(columns={'to':'toAddress'}, inplace=True)
transactionsFromUsersDex.rename(columns={'from':'fromAddress_'}, inplace=True)
transactionsFromUsersDex['fromAddress'] = transactionsFromUsersDex['fromAddress'].str.upper()
transactionsFromUsersDex['fromAddress_'] = transactionsFromUsersDex['fromAddress_'].str.upper()
transactionsFromUsersDex['toAddress'] = transactionsFromUsersDex['toAddress'].str.upper()

address = []
name = []
for index in range(0, ethData.getNumberContractAddressMap()):
    dex = ethData.getContractAddressMap(index)
    address.append(dex[1].upper())
    name.append(dex[0].upper())
dfDex = pd.DataFrame({'DEX_Address' : address, 'DEX_Name' : name})
dfDex.to_csv(r'DEX.csv')

base = '{0}'.format(10 ** 18) # = 10 ^ 18
sql = 'Select a.*, CASE WHEN fromAddress = fromAddress_ then "OUT" else "IN" END AS IN_OUT from transactionsFromUsersDex a'
df = psql.sqldf(sql)
dexList = []
caseWhenStatement_IN = []
caseWhenStatement_OUT = []
sumCaseWhenStatement = []
for dexIndex in range(0, ethData.getNumberContractAddressMap()):
    name = ethData.getContractAddressMap(dexIndex)[0]
    address = ethData.getContractAddressMap(dexIndex)[1]
    df[name + '_IN'] = np.where(df['fromAddress'] == address.upper(), True, False)
    dexList.append(name + '_IN')
    df[name + '_OUT'] = np.where(df['toAddress'] == address.upper(), True, False)
    dexList.append(name + '_OUT')
    caseWhenStatement_IN.append("WHEN {0} = True THEN 'SUM_{0}'".format(name + '_IN'))
    caseWhenStatement_OUT.append("WHEN {0} = True THEN 'SUM_{0}'".format(name + '_OUT'))
    sumCaseWhenStatement.append("SUM(CASE WHEN {0} = True THEN 1 ELSE 0 END) AS CNT_{0}, SUM(CASE WHEN {0} = True THEN value / {1} ELSE 0 END) AS SUM_{0}_ETH".format(name + '_OUT', base))
dexList = ", ".join(dexList)
caseWhenStatement_IN = " ".join(caseWhenStatement_IN)
caseWhenStatement_OUT = " ".join(caseWhenStatement_OUT)
sumCaseWhenStatement = ", ".join(sumCaseWhenStatement)

sql = 'With toDexTransactions As ' \
      '(Select a.* ' \
      'From df a inner join dfDex b ' \
      'on (a.toAddress = b.DEX_Address or a.fromAddress = b.DEX_Address)) ' \
      'Select fromAddress, count(*) as CNT_DEX_TX, ' \
      'SUM(CASE WHEN IN_OUT = "OUT" THEN value / {1} END) as TOTAL_DEX_OUT_ETH, ' \
      'MIN(blockNumber) as MIN_BLOCK_ANALYZED, ' \
      'MAX(blockNumber) as MAX_BLOCK_ANALYZED, ' \
      '(MAX(blockNumber)-MIN(blockNumber)) / count(*) as FREQ_CNT_DEX_TX, ' \
      'MAX(CASE WHEN IN_OUT = "OUT" THEN value / {1} END) as MAX_DEX_OUT_ETH, ' \
      '{0} ' \
      'From toDexTransactions ' \
      'Group by fromAddress ' \
      'Order By TOTAL_DEX_OUT_ETH desc'.format(sumCaseWhenStatement, base)

result = psql.sqldf(sql)

q = [.5, .65, .8, .9, .95, .995]
totalDex_OUT_ETH_q = np.quantile(result['TOTAL_DEX_OUT_ETH'] , q, axis = 0)
plot_df = pd.DataFrame(totalDex_OUT_ETH_q, columns = ['TOTAL_DEX_OUT_ETH_Q'])
plot_df['QUANTILE'] = pd.Series(q)
sql = "Select *, " \
      "CASE " \
      "WHEN TOTAL_DEX_OUT_ETH >= 0 AND TOTAL_DEX_OUT_ETH <= {0} THEN 'MINIMAL' " \
      "WHEN TOTAL_DEX_OUT_ETH > {0} AND TOTAL_DEX_OUT_ETH <= {1} THEN 'LOW' " \
      "WHEN TOTAL_DEX_OUT_ETH > {1} AND TOTAL_DEX_OUT_ETH <= {2} THEN 'BRONZE' " \
      "WHEN TOTAL_DEX_OUT_ETH > {2} AND TOTAL_DEX_OUT_ETH <= {3} THEN 'SILVER' " \
      "WHEN TOTAL_DEX_OUT_ETH > {3} AND TOTAL_DEX_OUT_ETH <= {4} THEN 'GOLD' " \
      "WHEN TOTAL_DEX_OUT_ETH > {4} AND TOTAL_DEX_OUT_ETH <= {5} THEN 'PLATINUM' " \
      "WHEN TOTAL_DEX_OUT_ETH > {5} THEN 'PRIVILEGE' " \
      "END AS ADDRESS_CATEGORY " \
      "FROM result".format(totalDex_OUT_ETH_q[0], totalDex_OUT_ETH_q[1], totalDex_OUT_ETH_q[2], totalDex_OUT_ETH_q[3], totalDex_OUT_ETH_q[4], totalDex_OUT_ETH_q[5])
result = psql.sqldf(sql)
result.to_csv(r"export/CUSTOMER_DEX_ANALYSIS_{0}.csv".format(datetime.now().strftime("%Y%m%d_%H%M%S")))

In [ ]:
#connectionString = "http://127.0.0.1:7545"
connectionString = 'https://mainnet.infura.io/v3/9a82d6f712b6474eb712fa507d110055'

In [ ]:
core = AIDeFiCore.ethData()

In [ ]:
w3 = core.blockchainConnection(connectionString)

In [ ]:
w3.isConnected()

In [ ]:
lastBlockNumber = core.getLastBlockNumber(w3)
(df_block, df_transaction, df_transactionReceipt) = core.importBlockInterval(w3, lastBlockNumber-1, lastBlockNumber)

In [ ]:
df_block.to_csv("export/df_block.csv")
df_transaction.to_csv("export/df_transaction.csv")
df_transactionReceipt.to_csv("export/df_transactionReceipt.csv")

In [ ]:
ai = AIDeFiCore.AI()

In [ ]:
transactionsFromUsersDex = pd.from_csv('export/CUSTOMERS_TRANSACTIONS_FROM_DEX/filename.csv')
transactionsFromUsersDex.set_index(transactionsFromUsersDex['from'])
customerList = pd.from_csv('export/CUSTOMERS_LIST_FROM_DEX.csv')
customerList.set_index(customerList['from'])
transactionsFromUsersDex.join(customerListl, suffix = '_caller', rsuffix = '_other')
transactionsFromUsersDex['IN_OUT'] = np.where(transactionsFromUsersDex['from_caller'].isnan(), 'OUT', 'IN')
transactionsFromUsersDex